# Data Wrangling Homework

We have three files that describe some data of mobile phones.
- Mobile data 1.csv: This file contains
    - id - ID
    - battery_power - Total energy a battery can store in one time measured in mAh
    - blue - Has bluetooth or not
    - clock_speed - speed at which microprocessor executes instructions
    - dual_sim - Has dual sim support or not
    - fc - Front Camera mega pixels
    - four_g - Has 4G or not
    - int_memory - Internal Memory in Gigabytes
    - m_dep - Mobile Depth in cm
    - mobile_wt - Weight of mobile phone
    - n_cores - Number of cores of processor
    - pc - Primary Camera mega pixels
    - px_height - Pixel Resolution Height
    - px_width - Pixel Resolution Width
    - ram - Random Access Memory in Megabytes
    - sc_h - Screen Height of mobile in cm
    - sc_w - Screen Width of mobile in cm
    - talk_time - longest time that a single battery charge will last when you are
    - three_g - Has 3G or not
    - touch_screen - Has touch screen or not
    - wifi - Has wifi or not
    - price_range - This is the target variable with value of 0(low cost), 1(medium cost), 2(high cost) and 3(very high cost).
- Mobile data 2.csv: This file contains
    - ID	
    - Brand	- the make of the phone, e.g., Apple, Samsunig, Ericsson, etc.
    - Phone	- the model fo the phone, e.g. iPhone 12
    - Picture URL small - a hyperlink to an image of the phone
    - Body Dimensions - this is compound value that is not uniform. It contains  w x h x d in both mm and inches
    - Body Weight - the information is given also in grams and lb but also for several configurations
    - Display Resolution - this column combines different values of w x h pixels, screen ratio and pixel density
- Price ranges.csv - simple file that contains the range id and the min and max value for each range

### Our objective is to merge the data into one data set that looks like the one below.

In [292]:
## DO NOT RUN ME

,battery_power,clock_speed,fc_megapixel,int_memory,m_depth,mobile_wt,n_cores,pc_megapixel,px_height,px_width,ram,sc_h,sc_w,talk_time,id,phone_id,bluetooth,dual_sim,four_g,three_g,touch_screen,wifi,price_range,ID,Brand,Phone,BodyDimensions,BodyWeight,DisplayResolution,WeightInGrams,Config,Weight,Height,Width,Depth,price_range_r,Min,Max
19,682,0.5,4.0,19,1.0,121.0,4,11,902,1064,2337.0,11,1,18,20,115,1,0,0,0,1,1,1,115,Ericsson,Ericsson T10s,105 x 49 x 24 mm (4.13 x 1.93 x 0.94 in);,135 g (4.76 oz);,"101 x 33 pixels, 3 x 12 chars;",135,Standard,135,105,49,24,1,501,700
22,1949,2.6,4.0,47,0.3,199.0,4,7,407,822,1433.0,11,5,20,23,118,0,1,0,0,0,1,1,118,Ericsson,Ericsson T28 World,97 x 50 x 15 mm (3.82 x 1.97 x 0.59 in);,83 g (2.93 oz);,"101 x 33 pixels, 3 x 12 chars;",83,Standard,83,97,50,15,1,501,700
42,1253,0.5,5.0,5,0.2,152.0,2,19,685,714,1878.0,15,0,4,43,2826,1,1,1,1,1,0,1,2826,Apple,Apple iPhone 3GS,115.5 x 62.1 x 12.3 mm (4.55 x 2.44 x 0.48 in);,135 g (4.76 oz);,"320 x 480 pixels, 3:2 ratio (~165 ppi density);",135,Standard,135,115.5,62.1,12.3,1,501,700
43,1656,1.0,5.0,34,0.1,166.0,3,7,880,1456,1629.0,15,12,14,44,3275,0,0,1,1,1,0,1,3275,Apple,Apple iPhone 4,115.2 x 58.6 x 9.3 mm (4.54 x 2.31 x 0.37 in);,137 g (4.83 oz);,"640 x 960 pixels, 3:2 ratio (~330 ppi density);",137,Standard,137,115.2,58.6,9.3,1,501,700
45,1514,2.9,0.0,27,0.2,118.0,3,1,186,1810,1152.0,8,3,20,46,3827,0,0,0,0,1,1,1,3827,Apple,Apple iPad Wi-Fi + 3G,242.8 x 189.7 x 13.4 mm (9.56 x 7.47 x 0.53 in);,730 g (1.61 lb);,"768 x 1024 pixels, 4:3 ratio (~132 ppi density);",730,Standard,730,242.8,189.7,13.4,1,501,700


Notably, some renaming will be necessary to obtain the same schema.

#### Is it possible as they are?

### First, we have to make some pre-processing on some of the individual files

For instance, we need to make the following pre-processing (transformations) on  mobile data 1.csv,
- Remove duplicates
- Rename columns
- Handle missing data (this will be revisited with more details when talking about data cleansing)
 - Just drop the rows
 - Fill with some neighbor value
 - Fill with some statistical value (mean or median)
- Handling outliers
- Standardizing the data

For mobile data 2.csv, we need to make the following pre-processing (transformations)

- Out of the Body dimensions column, we need to extract three columns: Width, Height, Depth all in mm. **This is a one to one transformation.**
- Out of the Body weight, we need to extract the weight in grams for each possible offering. For example, for iPads, ones that come with WiFi only have a different weight different from those that come with additional 4G support. **This is a one to many transformation**

In [ ]:
%config Completer.use_jedi = False #Enable Autocompletion

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns # better visualization tool
from sklearn.preprocessing import StandardScaler
%matplotlib inline


In [ ]:
pd

## Pre-processing the first data set

### Profiling
First we do some generic profiling. We check the shape, the data types, the columns and a general statistical description of the data.

In [297]:
# load mobile_data_1.csv in a pandas dataframe
data_1 = pd.read_csv('./data/input/mobile_data_1.csv')

In [298]:
data_1.head()

,id,phone_id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,1,96,842,0,2.2,0,1.0,0,7,0.6,188.0,2,2,20,756,2549.0,9,7,19,0,0,1,1
1,2,97,1021,1,0.5,1,NaN,1,53,0.7,136.0,3,6,905,1988,NaN,17,3,7,1,1,0,2
2,3,98,563,1,0.5,1,2.0,1,41,0.9,NaN,5,6,1263,1716,2603.0,11,2,9,1,1,0,2
3,4,99,615,1,2.5,0,NaN,0,10,0.8,131.0,6,9,1216,1786,2769.0,16,8,11,1,0,0,2
4,5,100,1821,1,1.2,0,13.0,1,44,NaN,141.0,2,14,1208,1212,1411.0,8,2,15,1,1,0,1


In [ ]:
# check amount of rows and columns
data_1.shape

In [ ]:
# check their types

In [ ]:
data_1.dtypes

In [ ]:
# Observe the dataset main statistics (visualize it)

In [ ]:
data_1.describe()

In [ ]:
# check how much memory does the dataframe consume
data_1.info()
# if column contains "object" data types, use .info(memory_usage='deep')

In [ ]:
# explore the datasets first and last elements

In [ ]:
data_1.head()

In [ ]:
data_1.columns

In [ ]:
# NB - notice when you overwrite the existing dataframe, or when are you creating a new df
data_1 = data_1.rename(columns = {'blue' : 'bluetooth', 
                              'fc' : 'fc_megapixel',
                              'pc' : 'pc_megapixel',
                              'm_dep' : 'm_depth'})

In [ ]:
data_1.head()

In [ ]:
# modify display behaviour
pd.get_option('display.max_columns')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
data_1.head()

In [ ]:
data_1.sample(100)

### Handling duplicates

In [ ]:
# remove douplicates from the dataset.

In [ ]:
dupes = 

In [ ]:
dupes.head()

In [ ]:
data_1 = 

### Handling Missing Data

In [ ]:
# Identify what fields have null values (count)

You should have identified different columns, i.e., fc_megapixel, m_depth, mobile_wt, ram.
Each requires a different method to be repaired. The colum is indicated below.

#### Replacing Nans with standard values (0) for (fc_megapixel)
_data imputation_

In [ ]:
data_1['<<Column Name>>'] = 

data_1.isnull().sum()

#### Filling Forward or Backward (ram)
If we supply a method parameter to the fillna() method, we can fill forward or backward as we need. To fill forward, use the methods pad or fill, and to fill backward, use bfill and backfill.

NB! Make sure this makes sense for your data.

In [ ]:
data_1['<<Column Name>>'] = 



#### Replacing nan with median of the column (mobile_wt)

In [ ]:
data_1['<<Column Name>>'] = 

#### Dropping Nans (m_depth)
Could we do this before?


In [ ]:
data_1 = 

In [ ]:
data_1.isnull().sum()

In [ ]:
data_1.shape

### Handling outliers
Another type of profiling is to check for legitimate values and handling outliers. 
We need to do that with numerical values only, i.e., drop the non-numerical columns, but save them for rejoining them late

In [ ]:
numerical_data = 
numerical_data.head()

In [ ]:
categorical_data = 
categorical_data.head()

In [ ]:
# A common way to visualize outliers is using boxplots. Explore by observing the "ram" column using seaborn (sns)

In [ ]:
# view a single column's outliers
sns.boxplot(data=numerical_data['ram'], orient = 'v')

In [ ]:
# view all columns, again using seaborn (sns)
bp = sns.boxplot(data = numerical_data)
bp.set_xticklabels(bp.get_xticklabels(), rotation=90)

To better visualize, you may want to standardize the values of the columns. You can use StandardScaler in sklearn.preprocessing

In [ ]:
scaler = StandardScaler()
scaled_array = scaler.fit_transform(numerical_data)

In [ ]:
scaled_data = 

In [ ]:
scaled_data.head()

In [ ]:
scaled_data.describe()

In [ ]:
#Plot the scaled data

#### Distinguish Outliers using IQR

The distance between the ﬁrst and third quartiles is a simple measure of spread that gives the range covered by the middle half of the data. This distance is called the interquartile range (IQR) and is deﬁned as

IQR = Q3 − Q1

<p float="right"><img src="./quartiles.png" width="300"/><img src="./box.png" width="100"/></p>


Any object that is more than 1.5 × IQR smaller than Q1 or 1.5 × IQR larger than Q3 is treated as an outlier because the region between Q1 − 1.5 × IQR and Q3 + 1.5 × IQR contains 99.3% of the objects. The rationale is similar to using 3σ as the threshold for normal distribution.


In [ ]:
# we can use Interquartile range (IQR) to distinguish outliers 
Q1 = 
Q3 = 

IQR = Q3 - Q1

print(IQR)

In [ ]:
outliers_removed_data = 

outliers_removed_data.shape

In [ ]:
#Show the absence of outliers (boxplot)

### Joing data back together

In [ ]:
final_data_1 = 
final_data_1.head()

### Write this part of the data to disk

In [ ]:
final_data_1.to_csv('./data/output/mobile_data_1_cleaned.csv', index = False)

## Pre-processing the second data set

In [ ]:
data_2 = 

In [ ]:
data_2.head(30)

We can observe that some of the columns have compound information, e.g. Body Dimension and Body Weight. Also, some records have meaningless data. Also, some columns are irrelevant to the analysis, like the Picture url.
So, we will do the following.
- Drop irrelevant columns, e.g. Picture url
- Drop records with meaningless data. We can drop records for which columns like Body dimensions, Body weight and display resolution are less than 20 characters long.
- Minor: some columns need renaming to meet the final schema (see above)



In [ ]:
data_2 = #Dropping

In [ ]:
data_2 = # Renaming

In [ ]:
data_2.info(memory_usage='deep') #

We can observe that Pandas is not able to infer the type on its own for the non-ID columns. 

In [ ]:
# Enforce the type str where is needed.

In [ ]:
data_2['Brand']= 
data_2['Phone']= ...
#MORE?

In [ ]:
# Remove data with missing information on Body Weight and Display Res (See below)

,ID,Brand,Phone,BodyDimensions,BodyWeight,DisplayResolution
0,96,Ericsson,Ericsson GH 218,130 x 49 x 36 mm (5.12 x 1.93 x 1.42 in);,;,1 line;


In [ ]:
meaningful_data_2 = 

In [ ]:
meaningful_data_2.head()

In [ ]:
meaningful_data_2.shape 

We need to extract the weight as a new column for the value of grams only.

In [ ]:
# split the string describing the weight by the **g** character. You can use "apply" to split the data

In [ ]:
WeightInGrams = 

In [ ]:
WeightInGrams

In [ ]:
# Before assigning the separated data to the df, make a copy to avoid circular reference and/or making unwanted changes in previous dataframes
meaningful_data_3 = meaningful_data_2.copy()

In [ ]:
meaningful_data_3['WeightInGrams'] = WeightInGrams 

In [ ]:
meaningful_data_3.shape

In [ ]:
meaningful_data_3.sample(100)

We can notice in the original data that the ''BodyWeight'' column has actually multiple values. For example, we can find value like _331 g (Wi-Fi) / 341 g (3G/LTE) (11.68 oz);_. This means that there are different weights for different configurations. Our objective is to create a new row for each different weight and put another column called configuration.

One way is to first get those rows that have multiple weight values (split and count)

In [ ]:
multiple_weight_phones = 

In [ ]:
multiple_weight_phones.shape

In [ ]:
multiple_weight_phones

What is wrong with the separating character. Sometimes is ',' sometimes is '/'. Can we uniform it before splitting? Also, we have another occurrence of '/' in some configurations e.g., (3G/LTE), which will complicate the splitting even further. Notably, the relevant occurrence of weight separators is followed by a space. Can we replace '/ ' with '//'. Hint: Use apply

In [ ]:
unifiedSeparator = #Replace the values

In [ ]:
unifiedSeparator.head()

In [300]:
# Drop columns we no longer need.

In [ ]:
multiple_weight_phones = 

In [ ]:
multiple_weight_phones['BodyWeight'] = unifiedSeparator
multiple_weight_phones.shape

In [ ]:
multiple_weight_phones['BodyWeight']

In [ ]:
# Split the new added column

In [ ]:
multiple_weight_2 = 
# Careful! this is one of the few operation that does not cleare a dataframe, create one with a column named 'Config'
multiple_weight_2 = 
multiple_weight_2

In [ ]:
multiple_weights_3 = # Join data back together
multiple_weights_3.head()

In [ ]:
multiple_weights_3.columns

In [ ]:
multiple_weights_3 = # drop useless columns
multiple_weights_3.head()

Now, we can split the config column to weight and config, again using apply

In [ ]:
weight = #split
weight.head()

In [ ]:
config = #split
config.head()

In [ ]:
multiple_weights_3= # Drop config
multiple_weights_3.columns

In [ ]:
multiple_weights_3['Config'] = config

In [ ]:
multiple_weights_3['Weight'] = weight 

We can finally join data back together , we can add a right suffix to distinguish the join key rsuffix='_multiple'

In [ ]:
meaningful_data_4 = # left join back together 

In [ ]:
meaningful_data_4.sample(10)

In [ ]:
meaningful_data_4 = # Drop the suffixed ID

In [ ]:
meaningful_data_4.sample(10)

In [ ]:
meaningful_data_4['Weight']# We can fille the values in this column with those in grams. 

In [ ]:
meaningful_data_4.sample(50)

In [ ]:
meaningful_data_4 = # Drop the suffixed column

In [ ]:
meaningful_data_4.head(20)

Checking the dataframes held in memory

In [ ]:
%whos DataFrame

In [ ]:
# Find the memory footprint
df_list = []    
for var in dir():
    if isinstance(locals()[var], pd.core.frame.DataFrame) and var[0] != '_':
        df_list.append(var)    

In [ ]:
memory_cons = 0
for d in df_list:
    memory_cons += locals()[d].memory_usage(deep=True).sum()

print(f'Total memory consumed by dataframes: {round(memory_cons/1024/1024,1)} MB')

#### Few More Steps
 

In [ ]:
# Fill the empty "Config" column in the meaningful_data_4 dataframe with value "Standard"

config_values = 
meaningful_data_5 = meaningful_data_4.copy() # make a copy like before


In [ ]:
# Transform the body dimensions and get separate height, width, and depth dimensions in mm

dimensions = # strip and split
dimensions.head()

In [ ]:
height_dim = 
width_dim =
depth_dim = 

In [ ]:
height_dim.head()
#width_dim.head()
#depth_dim.head()

In [ ]:
final_data_2 = # Join everything back together

In [ ]:
final_data_2.head()

In [ ]:
# Inner join the three data sets:
## preprocessed mobile 1 dataset
## preprocessed mobile 2 dataset
## price ranges

joined_data.head()

In [294]:
joined_data.dtypes

battery_power          int64
clock_speed          float64
fc_megapixel         float64
int_memory             int64
m_depth              float64
mobile_wt            float64
n_cores                int64
pc_megapixel           int64
px_height              int64
px_width               int64
ram                  float64
sc_h                   int64
sc_w                   int64
talk_time              int64
id                     int64
phone_id               int64
bluetooth              int64
dual_sim               int64
four_g                 int64
three_g                int64
touch_screen           int64
wifi                   int64
price_range            int64
ID                     int64
Brand                 object
Phone                 object
BodyDimensions        object
BodyWeight            object
DisplayResolution     object
WeightInGrams         object
Config                object
Weight                object
Height                object
Width                 object
Depth         

In [295]:
joined_data.shape

(560, 38)

In [ ]:
# Save the final data frame in a file 'ready_for_analysis.csv'

joined_data.to_csv('./data/output/ready_for_analysis.csv')

#### Check also:

* <a href="https://pandas.pydata.org/docs/reference/index.html">API reference</a>
* <a href="https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf">Official cheat sheet</a>